In [28]:
#https://medium.com/@viniciusqroz/using-keras-tuner-to-find-the-best-parameters-for-your-neural-network-model-2dc02e0a1203

In [29]:
!pip install keras-tuner

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [31]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()

In [32]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first dense layer
    # Choose an optimal value between 32-512
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units1, activation='relu'))

    # Tune the number of units in the second dense layer
    # Choose an optimal value between 32-512
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units2, activation='relu'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [33]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

Reloading Tuner from my_dir/helloworld/tuner0.json


In [34]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))


Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
32                |352               |units1
512               |352               |units2
0.01              |0.01              |learning_rate

Epoch 1/5
1745/1875 [==========================>...] - ETA: 0s - loss: 2.5770 - accuracy: 0.1018

KeyboardInterrupt: 

In [ ]:
best_models = tuner.get_best_models(num_models=1)

In [ ]:
test_loss, test_acc = best_models[0].evaluate(x_test, y_test)
print('Test accuracy:', test_acc)